#1. Import Libraries

In [2]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
    auc
)
import matplotlib.pyplot as plt
from scipy.stats import expon, reciprocal
from sklearn.metrics import classification_report
import seaborn as sns

# 2. Load Data

In [3]:
drive.mount('/content/drive')


unmon_features= pd.read_csv('/content/drive/MyDrive/MLproject/unmon_features.csv')
mon_features= pd.read_csv('/content/drive/MyDrive/MLproject/mon_features.csv')
mon_labels = pd.read_csv('/content/drive/MyDrive/MLproject/mon_labels.csv')


Mounted at /content/drive


In [4]:
mon_features['label'] = mon_labels.values
unmon_features['label'] = -1

combined_data = pd.concat([mon_features, unmon_features], axis=0, ignore_index= True)
combined_data.fillna(0, inplace=True)

X = combined_data.drop('label', axis=1)
y = combined_data['label']


print(X.head)

<bound method NDFrame.head of        total_num  total_sum_dir  total_avg  inpkt_num  inpkt_avg  inpkt_sum  \
0           1421        -603648   4.315749       1300   4.292023    5579.63   
1            518        -183296   4.611815        438   4.673242    2046.88   
2           1358        -574464   5.488969       1240   5.459452    6769.72   
3           1446        -615424   5.995512       1324   5.948693    7876.07   
4           1406        -602112   4.698869       1291   4.663873    6021.06   
...          ...            ...        ...        ...        ...        ...   
28995       4593       -1928704  12.494690       4180  12.360713   51667.78   
28996       5110       -2158592  13.844609       4663  13.729719   64021.68   
28997        361        -124416   3.568144        302   3.251291     981.89   
28998        509        -162304   6.892377        413   6.689346    2762.70   
28999       9990       -4785152   7.916714       9668   7.923616   76605.52   

       inpkt_num_frac

# Preprocess Data

In [5]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=50
)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# 4. Model selection

In [6]:
# Train SVM model
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train_scaled, y_train)

# Evaluate model on test data
y_pred = svm_model.predict(X_test_scaled)

# Calculate basic evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")


Accuracy: 0.4578
Confusion Matrix:
[[1951    1    0 ...    0    0    0]
 [  23    1    1 ...    0    0    0]
 [  29    0   17 ...    0    0    0]
 ...
 [  40    0    0 ...    0    0    0]
 [  15    0    0 ...    0   21    0]
 [  46    0    0 ...    0    0    0]]


# 5. Hyperparameter Tuning

In [8]:
# Define hyperparameter grid for GridSearchCV
param_grid = {
    'C': [100, 200, 300],
    'gamma': [0.1, 0.3],
    'kernel': ['rbf']
}

# Perform GridSearchCV
grid_search = GridSearchCV(SVC(), param_grid, cv=3, scoring='accuracy', verbose=2)
grid_search.fit(X_train_scaled, y_train)

# Retrieve the best model
best_params = grid_search.best_params_
best_svm_model = grid_search.best_estimator_
print(f"Best parameters (GridSearchCV): {best_params}")

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time=  47.9s
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time=  43.6s
[CV] END .......................C=100, gamma=0.1, kernel=rbf; total time=  43.6s
[CV] END .......................C=100, gamma=0.3, kernel=rbf; total time=  48.1s
[CV] END .......................C=100, gamma=0.3, kernel=rbf; total time=  49.8s
[CV] END .......................C=100, gamma=0.3, kernel=rbf; total time=  49.0s
[CV] END .......................C=200, gamma=0.1, kernel=rbf; total time=  45.3s
[CV] END .......................C=200, gamma=0.1, kernel=rbf; total time=  45.8s
[CV] END .......................C=200, gamma=0.1, kernel=rbf; total time=  44.5s
[CV] END .......................C=200, gamma=0.3, kernel=rbf; total time=  47.9s
[CV] END .......................C=200, gamma=0.3, kernel=rbf; total time=  48.9s
[CV] END .......................C=200, gamma=0.3,

In [9]:
y_pred = best_svm_model.predict(X_test_scaled)
# Evaluation with the best parameters
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with best parameters: {accuracy:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(95, 95))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

Accuracy with best parameters: 0.7090

Classification Report:
               precision    recall  f1-score   support

          -1       0.73      0.86      0.79      1991
           0       0.42      0.43      0.43        30
           1       0.79      0.58      0.67        52
           2       0.70      0.80      0.75        41
           3       0.71      0.67      0.69        43
           4       0.57      0.67      0.62        36
           5       0.77      0.66      0.71        35
           6       0.63      0.86      0.73        28
           7       0.62      0.66      0.64        35
           8       0.83      0.56      0.67        36
           9       0.63      0.78      0.70        41
          10       0.73      0.63      0.68        38
          11       0.76      0.72      0.74        36
          12       0.80      0.80      0.80        41
          13       0.33      0.35      0.34        34
          14       0.57      0.47      0.51        43
          15      

In [10]:
# Define hyperparameter distributions for RandomizedSearchCV
param_distributions = {
    'C': reciprocal(1, 1000),
    'gamma': expon(scale=0.1),
    'kernel': ['rbf'],
    'class_weight': ['balanced', None]
}

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(SVC(), param_distributions, n_iter=50, cv=2, scoring='accuracy', verbose=2, random_state=50, n_jobs=-1)
random_search.fit(X_train_scaled, y_train)

# Retrieve the best model
best_params_random = random_search.best_params_
best_svm_model_random = random_search.best_estimator_
print(f"Best parameters (RandomizedSearchCV): {best_params_random}")

Fitting 2 folds for each of 50 candidates, totalling 100 fits
Best parameters (RandomizedSearchCV): {'C': 218.69176239528977, 'class_weight': None, 'gamma': 0.06980161530282498, 'kernel': 'rbf'}


In [18]:
# Predict using the best model from RandomizedSearchCV
y_pred_random = best_svm_model_random.predict(X_test_scaled)

# Evaluate model performance
accuracy_best = accuracy_score(y_test, y_pred_random)
conf_matrix_best = confusion_matrix(y_test, y_pred_random)
precision = precision_score(y_test, y_pred_random, average='micro')  # 'macro'로 설정
recall = recall_score(y_test, y_pred_random, average='macro')        # 'macro'로 설정

# Calculate ROC and Precision-Recall AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_random, pos_label=1)
roc_auc = auc(fpr, tpr)
precision_values, recall_values, _ = precision_recall_curve(y_test, y_pred_random, pos_label=1)
pr_auc = auc(recall_values, precision_values)
# Display results
print(f"Model Accuracy: {accuracy_best:.4f}")
print(f"Model Precision: {precision:.4f}")
print(f"Model Recall: {recall:.4f}")
print(f"Model ROC AUC: {roc_auc:.4f}")
print(f"Model PR AUC: {pr_auc:.4f}")
print(f"Model Confusion Matrix:")
plt.figure(figsize=(95, 95))
sns.heatmap(conf_matrix_best, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


Model Accuracy: 0.6993
Model Precision: 0.6993
Model Recall: 0.6254
Model ROC AUC: 0.4105
Model PR AUC: 0.0071
Model Confusion Matrix:


In [19]:
# Plot ROC and Precision-Recall curves
plt.figure(figsize=(12,6))

# ROC Curve
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")

# Precision-Recall Curve
plt.subplot(1, 2, 2)
plt.plot(recall_values, precision_values, color='blue', lw=2, label=f'PR curve (area = {pr_auc:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")

plt.tight_layout()
plt.show()
